In [ ]:
from __future__ import print_function, division
import sys
sys.path.append('/home/ego/Github/david/')

import os
from os.path import exists, join, isfile

import dataset
import pandas as pd
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

from david.youtube import scraper, searchv2
from david.pipeline import TextMetrics, TextPreprocess

In [ ]:
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

class Substitution:
    """A decorator to take a function's docstring and perform string
    substitution on it. 
    This decorator should be robust even if func.__doc__ is None
    (for example, if -OO was passed to the interpreter)

    Usage: construct a docstring.Substitution with a sequence or
    dictionary suitable for performing substitution; then
    decorate a suitable function with the constructed object. e.g.

    >>> sub_author_name = Substitution(author='Jason')
    >>> ...
    >>> @sub_author_name
    >>> def some_function(x):
            "%(author)s wrote this function"

    # note that some_function.__doc__ is now "Jason wrote this function"
    One can also use positional arguments.

    >>> sub_first_last_names = Substitution('Edgar Allen', 'Poe')

    >>> @sub_first_last_names
    >>> def some_function(x):
            "%s %s wrote the Raven"
    """
    def __init__(self, *args, **kwargs):
        if args and kwargs:
            raise AssertionError("Only positional or keyword args are allowed")
        # which ever (args | kwargs) is passed thorugh the condition.
        self.params = args or kwargs

    def __call__(self, func: Callable) -> Callable:
        func.__doc__ = func.__doc__ and func.__doc__ % self.params
        return func

    def update(self, *args, **kwargs) -> None:
        """Update self.params with supplied args.
        If called, we assume self.params is a dict.
        """
        self.params.update(*args, **kwargs)

In [ ]:
sub_author_name = Substitution(author='Carlos', age=27, specs='Information Systems')
sub_author_name.params

In [ ]:
@sub_author_name
def func(x):
    "%(author)s function"
func.__doc__

In [ ]:
import numpy as np

data = {
    "col_{0:02d}".format(i): np.random.randint(0, high=1000, size=30000)
    for i in range(100)
}
data

In [ ]:
JSON = 'downloads/4Dk3jOSbz_0.json'

from pandas import RangeIndex

In [ ]:
RangeIndex??

In [ ]:
df = pd.DataFrame(JSON, index=RangeIndex(start=0, stop=100, step=1))
df.describe()

In [ ]:
class DFClassGenerator:
    '''Example Usage.
    comment = CommentsFrame(data=dict(
        author=['carlos', 'chucho'],
        cid=[122, 177],
        text=['this is carlos', 'chucho here'],
        time=['10:30', '19:40']))
    type(comment)
    '''
    CLASS_HEADER = 'class {class_name}(pd.DataFrame):'
    COLUMNS = '    {var} = "{label}"'   # we cheat an encode 4 spaces here,for demo

    CONSTRUCTOR =  ("    @property\n"
                    "    def _constructor(self):\n"
                    "        return {class_name}")

    @classmethod
    def generate_class(cls, df, class_name):
        cols = [cls.COLUMNS.format(var=c.upper(), label=c)
                for c in df.columns] # works for single hierarchical column index
        lines = [cls.CLASS_HEADER.format(class_name=class_name)]
        constructor = cls.CONSTRUCTOR.format(class_name=class_name)
        source_code = '\n'.join(lines + cols) + '\n\n' + constructor
        print(source_code)
        
#source_code = DFClassGenerator.generate_class(metric, 'CommentsFrame')
#print(source_code)

In [ ]:
class DavidFrame(pd.DataFrame):

    def __init__(self, *args, **kwargs):
        super(DavidFrame, self).__init__(*args, **kwargs)
        self._metadata = {'test': 'TEST'}
        
    def to_textfile(self, fn, text_col='text'):
        with open(fn, 'w', encoding='utf-8') as f:
            for x in self[text_col].tolist():
                if len(x) != 0:
                    f.write('%s\n' % x)
            f.close()

    @property
    def _constructor(self):
        print("_constructor called")
        return DavidFrame

    def __finalize__(self, other, method=None, **kwargs):
        '''Propagate metadata from other to self.
        '''
        print("__finalize__ called")
        for name in self._metadata:
            object.__setattr__(self, name, getattr(other, name, None))
        return self

    def copy(self, deep_copy=True):
        '''Make a copy of the class instance object.

        deep_copy :  makes a deep copy, i.e. also copy data and
        returns a copy of the obj.
        '''
        data = self._data
        if deep_copy:
            data = data.copy()
        return DavidFrame(data).__finalize__(self)

In [ ]:
def builder(func, *pargs, **kwards):
    print(f'calling: {func.__name__}')
    return func(*pargs, **kwards)

In [ ]:
metric = TextMetrics(json_fp='downloads/4Dk3jOSbz_0.json')
david = DavidFrame(metric.to_dict(orient='dict'))

In [ ]:
from readline import read_history_file, write_history_file, set_auto_history

In [ ]:
import atexit
import code
import os
import readline

class HistoryConsole(code.InteractiveConsole):
    def __init__(self, locals=None, filename="<console>",
                 histfile=os.path.expanduser("~/.console-history")):
        code.InteractiveConsole.__init__(self, locals, filename)
        self.init_history(histfile)

    def init_history(self, histfile):
        readline.parse_and_bind("tab: complete")
        if hasattr(readline, "read_history_file"):
            try:
                readline.read_history_file(histfile)
            except FileNotFoundError:
                pass
            atexit.register(self.save_history, histfile)

    def save_history(self, histfile):
        readline.set_history_length(1000)
        readline.write_history_file(histfile)

In [ ]:
hc = HistoryConsole()
hc

In [ ]:
# this actuaclly starts a terminal session lol
hc.interact()